In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
import datetime
from requests.exceptions import Timeout
import urllib.request
import requests


In [ ]:
url = 'https://www.smogon.com/stats/2024-02/gen9ou-1825.txt'
print('0')
res = requests.get(url,timeout=5)
html_page = res.content
print('1')
soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)
text = str(text)
print('2')
month_use = text.rsplit('\\n')
len(month_use)

In [ ]:

months = []
for i in range (2014,2025):
    for j in range(1,13):
        if j < 10:
            a = str(str(i)+'-0'+str(j))
        else:
            a = str(str(i)+'-'+str(j))
        months.append(a)
# changing the months to be updated to all the data we have
months = months[10:-1]
months = months[0:int(len(months)-9)]
len(months)


In [ ]:
# usages = []
# for i in range(0,len(months)):
#         print(months[i])
#         url = 'https://www.smogon.com/stats/'+months[i]+'/ou-1825.txt'
#         print('0')
#         res = requests.get(url,timeout=5)
#         html_page = res.content
#         print('1')
#         soup = BeautifulSoup(html_page, 'html.parser')
#         text = soup.find_all(text=True)
#         text = str(text)
#         print('2')
#         month_use = text.rsplit('\\n')
#         usages.append(month_use)
# print(usages)

In [ ]:
# data Scraping
usages = []
for i in range(0, len(months)):
    print(months[i])
    # these bozos at smogon decided to change the format of the usage stats
    if i < 32:
        url = 'https://www.smogon.com/stats/' + months[i] + '/ou-1825.txt'
    elif i >=32 and i <60 :
        url = 'https://www.smogon.com/stats/' + months[i] + '/gen7ou-1825.txt'
    elif i >=60 and i<96:
        url = 'https://www.smogon.com/stats/' + months[i] + '/gen8ou-1825.txt'
    elif i >=96:
        url = 'https://www.smogon.com/stats/' + months[i] + '/gen9ou-1825.txt'
    
    print('0')
    
    retries = 3  # Number of retries
    for attempt in range(retries):
        try:
            res = requests.get(url, timeout=0.5)
            res.raise_for_status()  # Raise an error for non-2xx responses
            
            html_page = res.content
            print('1')
            soup = BeautifulSoup(html_page, 'html.parser')
            text = soup.find_all(text=True)
            text = str(text)
            print('2')
            month_use = text.rsplit('\\n')
            usages.append(month_use)
            
            # Break out of the retry loop if successful
            break
        except Timeout:
            print(f"Attempt {attempt + 1}: Timeout occurred while fetching data from {url}")
            if attempt < retries - 1:
                print("Retrying...")
            else:
                print("Max retries exceeded.")
                # Append None to indicate failure
                usages.append(None)


In [ ]:
rank = [] 
name =[]
usage_perc =[]
raw =[]
raw_perc =[]
real =[]
real_perc=[]
period = []
for i in range(0,len(usages)):
    for j in range(0,len(usages[i])):
            a = months[i]
            
            spliter = usages[i][j].rsplit('|')
            if len(spliter)>1:
                period.append(a)
                rank.append(spliter[1])
                name.append(str.upper(spliter[2]))
                usage_perc.append( spliter[3])
                raw.append(spliter[4])
                raw_perc.append(spliter[5])
                real.append(spliter[6])
                real_perc.append(spliter[7])
print(len(period),len(rank))

In [126]:
df = pd.DataFrame({'Month':period,'Rank':rank,'Name':name,'Usage %':usage_perc,'Raw usage':raw,'Raw %':raw_perc,'Real usage':real,'Real %':real_perc})
df = df.iloc[1:len(df),:]

In [127]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.replace('', pd.NA, inplace=True)
df = df.replace('%','', regex=True)
        




C:\Users\Khush Trivedi\AppData\Local\Temp\ipykernel_32944\4269238271.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [128]:
df.to_csv('master_sheet.csv')
df

,Month,Rank,Name,Usage %,Raw usage,Raw %,Real usage,Real %
1,2014-11,1,LANDORUS-THERIAN,28.52431,213666,14.482,185662,15.866
2,2014-11,2,GRENINJA,25.03246,340569,23.084,255567,21.840
3,2014-11,3,HEATRAN,22.58069,206055,13.966,173494,14.826
4,2014-11,4,ROTOM-WASH,20.68535,235692,15.975,203429,17.384
5,2014-11,5,FERROTHORN,19.38253,264055,17.898,231488,19.782
...,...,...,...,...,...,...,...,...
80491,2024-02,760,FENNEKIN,0.00000,6,0.000,4,0.000
80492,2024-02,761,ROOKIDEE,0.00000,8,0.000,7,0.000
80493,2024-02,762,GOLETT,0.00000,5,0.000,5,0.000
80494,2024-02,763,SOBBLE,0.00000,7,0.000,6,0.000
